In [1]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import numpy as np
import pandas as pd
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return files, targets

files, targets = load_dataset('face_detector_images')
targets = targets.astype(int)
df = pd.DataFrame(targets, columns=['dog', 'human'])
df['file'] = pd.DataFrame(files)
df.to_pickle("saved_models/face_detector_images.pkl")

# print statistics about the dataset
print('The are %d images sourced from the human and dog image data sets.' % len(files))

df.head(5)

/home/ddonohue/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


The are 21584 images sourced from the human and dog image data sets.


,dog,human,file
0,1,0,face_detector_images/dog/Anatolian_shepherd_do...
1,0,1,face_detector_images/human/Robert_Bullock_0002...
2,1,0,face_detector_images/dog/Belgian_tervuren_0156...
3,0,1,face_detector_images/human/Sharon_Stone_0002.jpg
4,0,1,face_detector_images/human/Jean_Chretien_0033.jpg


In [2]:
# shuffle and split data into training and testing sets
X_train_valid, X_test, y_train_valid, y_test = train_test_split(df['file'], df.drop(['file'], axis=1), test_size=0.1, shuffle=True, random_state=7)
#X_train, X_test, y_train, y_test = train_test_split(df['file'], df['human'], test_size=0.1, shuffle=True, random_state=7)

# shuffle and split training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.111, shuffle=True, random_state=5)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.111, shuffle=True, random_state=5)

# print statistics about the dataset
print('There are a total %d images available for training.' % len(X_train))
print('There are a total %d images available for validation.' % len(X_valid))
print('There are a total %d images available for testing.' % len(X_test))

X_train = X_train[:8600]
y_train = y_train[:8600]
X_valid = X_valid[:1080]
y_valid = y_valid[:1080]
X_test = X_test[:1080]
y_test = y_test[:1080]

# I will be using about half of the training, validation, and testing sets after learning that my CPU and RAM 
# can't handle that much data
print('%d images will be used for training.' % len(X_train))
print('%d images will be used for validation.' % len(X_valid))
print('%d images will be used for testing.' % len(X_test))

There are a total 17268 images available for training.
There are a total 2157 images available for validation.
There are a total 2159 images available for testing.
8600 images will be used for training.
1080 images will be used for validation.
1080 images will be used for testing.


In [3]:
# Preprocessing of images before feeding into tensorflow CNN.

from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(X_train).astype('float32')/255
valid_tensors = paths_to_tensor(X_valid).astype('float32')/255
test_tensors = paths_to_tensor(X_test).astype('float32')/255

100%|██████████| 1080/1080 [00:05<00:00, 203.90it/s]


In [4]:
# Training, validation, and testing data have been exported to train model using Google Collaboratory notebook
# and take advantage of accelerated model fitting from cloud GPU.

import h5py

export_data = False
if export_data:
    h5f_xTrain = h5py.File('saved_models/train_tensors.h5', 'w')
    h5f_xTrain.create_dataset('train_tensors', data=train_tensors)
    h5f_xTrain.close()

    h5f_xValid = h5py.File('saved_models/valid_tensors.h5', 'w')
    h5f_xValid.create_dataset('valid_tensors', data=valid_tensors)
    h5f_xValid.close()

    h5f_xTest = h5py.File('saved_models/test_tensors.h5', 'w')
    h5f_xTest.create_dataset('test_tensors', data=test_tensors)
    h5f_xTest.close()

export_labels = False
if export_labels:
    h5f_yTrain = h5py.File('saved_models/y_train.h5', 'w')
    h5f_yTrain.create_dataset('y_train', data=y_train)
    h5f_yTrain.close()

    h5f_yValid = h5py.File('saved_models/y_valid.h5', 'w')
    h5f_yValid.create_dataset('y_valid', data=y_valid)
    h5f_yValid.close()

    h5f_yTest = h5py.File('saved_models/y_test.h5', 'w')
    h5f_yTest.create_dataset('y_test', data=y_test)
    h5f_yTest.close()

In [5]:
# Build CNN architecture.
# Architecture is adapted from CNN found here: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 64)        0         
__________

In [6]:
# Compile the model

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [15]:
# Train the model
# Model fitting was performed in Google Collaboratory notebook for speed improvements.

from keras.callbacks import ModelCheckpoint   


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.face_detector.hdf5', verbose=1, 
                               save_best_only=True)
model.fit(train_tensors, y_train, batch_size=100, epochs=1,
          validation_data=(valid_tensors, y_valid), callbacks=[checkpointer], 
          verbose=2, shuffle=True)

Train on 8600 samples, validate on 1080 samples
Epoch 1/1

Epoch 00001: val_loss improved from inf to 0.08208, saving model to saved_models/weights.best.face_detector.hdf5
 - 665s - loss: 0.4446 - acc: 0.8014 - val_loss: 0.0821 - val_acc: 0.9778


In [7]:
# Load the weights that yielded the best validation accuracy

model.load_weights('saved_models/weights.best.face_detector.hdf5')

In [8]:
# Evaluate and print test accuracy

score = model.evaluate(test_tensors, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.9990740740740741
